# This notebook is based on a CIKM workshop
https://github.com/terrier-org/cikm2021tutorial/blob/main/notebooks

In [ ]:
#learn more about pyterrier: https://pyterrier.readthedocs.io/en/latest/

In [ ]:
#install pyterrier
%pip install python-terrier

In [ ]:
%pip install nltk

In [34]:
import pyterrier as pt
import requests
import pandas as pd
import pickle
import nltk
import numpy as np

In [6]:
url =  "https://www.bibsonomy.org/json/search/artificial%20intelligence?items=1000"

In [7]:
res = requests.get(url)

In [8]:
data = res.json()

In [9]:
df_ai = pd.DataFrame(data["items"])
df_ai = df_ai[df_ai["type"] == "Publication"]

In [10]:
#drops columns where at least 50% of the rows have missing values
df_ai_filtered = df_ai.dropna(axis=1, thresh=len(df_ai)*0.5)

In [12]:
df_ai_filtered.head(10)

,type,id,tags,intraHash,label,user,description,date,changeDate,count,url,interHash,pub-type,year,author,authors,volume,pages,bibtexKey,publisher
1000,Publication,https://www.bibsonomy.org/bibtex/24e338b04bc2a...,[710-714],4e338b04bc2abafd9fa0a1f4fe79103a,Artificial Intelligence in Agriculture,ijtsrd,,2021-04-13 13:15:04,2021-04-13 13:15:04,1,https://www.ijtsrd.com/engineering/electrical-...,4d2f7caf67c19f0ff98f5cb8a04fb14d,article,2021,[Matthew N. O. Sadiku | Sarhan M. Musa | Abayo...,[{'first': 'Matthew N. O. Sadiku | Sarhan M. M...,5,710-714,conf/icegov/MalhotraA20,NaN
1001,Publication,https://www.bibsonomy.org/bibtex/29f3b59303571...,"[Artificial, Intelligence, Risks, and, Benefits]",9f3b593035714a894c265f4fb9c535e7,Artificial Intelligence Benefit and Risks,ijtsrd,,2020-05-14 10:40:49,2020-05-14 10:40:49,1,https://www.ijtsrd.com/computer-science/artifi...,ebd51221b66e0a22976b6bf55dcc8b89,article,2020,[Seeta M. Chauhan],"[{'first': 'Seeta M.', 'last': 'Chauhan'}]",4,1007-1009,noauthororeditor,NaN
1002,Publication,https://www.bibsonomy.org/bibtex/24bc08fb79365...,"[ANN, Fuzzylogic, ElectricalEngineering, Artif...",4bc08fb79365b5bc698af1cd97fc1a75,Artificial Intelligence in Power Station,ijtsrd,,2020-01-10 09:23:37,2020-01-10 09:23:37,1,https://www.ijtsrd.com/engineering/electrical-...,2b5bf53f5cc143129b642869065d06cc,article,2019,[P. Naveen | S. Nikitha | P. Sudeesh | V. Vais...,[{'first': 'P. Naveen | S. Nikitha | P. Sudees...,4,1020-1025,noauthororeditor,NaN
1003,Publication,https://www.bibsonomy.org/bibtex/29f87881e4dec...,"[Artificial, Intelligence, Markup, Chatbot, La...",9f87881e4decfde15a1af630c8ea27d5,Artificial Intelligence Based Training and Pla...,ijtsrd,,2019-03-26 12:35:44,2019-03-26 12:35:44,1,http://www.ijtsrd.com/computer-science/artific...,4fd6139b2f85d67ccf1a1943139555f0,article,2018,[Krishanu Deb | Pankaj Agrawal | Harish Nawale...,[{'first': 'Krishanu Deb | Pankaj Agrawal | Ha...,3,1057-1060,noauthororeditor,NaN
1004,Publication,https://www.bibsonomy.org/bibtex/263320f0ab96a...,"[Learning,, Systems,, Artificialintelligence, ...",63320f0ab96ae778b51b2376aa914f59,The Significance of Artificial Intelligence in...,ijtsrd,,2023-10-14 13:59:49,2023-10-14 13:59:49,1,https://www.ijtsrd.com/computer-science/artifi...,ce9618fb026d6db7108c7254d417554a,article,2023,[Dr. Atul Kumar Mishra],"[{'first': 'Dr. Atul Kumar', 'last': 'Mishra'}]",7,786-788,noauthororeditor,NaN
1005,Publication,https://www.bibsonomy.org/bibtex/2f76cb3964bef...,[explanation],f76cb3964befb565e8456efb255b9918,Explanation in artificial intelligence: Insigh...,brusilovsky,Explanation in artificial intelligence: Insigh...,2023-06-05 00:07:52,2023-06-05 00:07:52,2,https://www.sciencedirect.com/science/article/...,924bf61b21094d9c52e30f705ff72365,article,2019,[Tim Miller],"[{'first': 'Tim', 'last': 'Miller'}]",267,1-38,MILLER20191,NaN
1006,Publication,https://www.bibsonomy.org/bibtex/219d04dda236d...,"[simulation, ai]",19d04dda236da5a4e7592a9eb2d4f5eb,A framework for Integrating Artificial Intelli...,mardukasoka,A framework for Integrating Artificial Intelli...,2017-04-29 16:53:32,2017-04-29 16:53:32,2,http://dx.doi.org/10.1007/BF00851350,7f175ce43cdc3c37efa038de41588165,article,1994,"[Georgios I. Doukidis, Marios C. Angelides]","[{'first': 'Georgios I.', 'last': 'Doukidis'},...",8,55--85,Doukidis1994,NaN
1007,Publication,https://www.bibsonomy.org/bibtex/28a86ca74e262...,"[(AI),, Behavior,, Intelligent, Artificialinte...",8a86ca74e262b9883c8dd4a7c60f1652,Artificial Intelligence Role in Modern Science...,ijtsrd,,2024-01-05 10:08:27,2024-01-05 10:08:27,1,https://www.ijtsrd.com/computer-science/artifi...,1c66e4c94b299521b1f28370ecc7d992,article,2023,[Manish Verma],"[{'first': 'Manish', 'last': 'Verma'}]",7,335-342,journals/orl/LeiAG93,NaN
1008,Publication,https://www.bibsonomy.org/bibtex/2b31f13f50e07...,"[Human, Intelligence,, Creativity,, Artificial...",b31f13f50e07ac7e22de4ea993ba89ee,Artificial Intelligence Catalyzes a Revolution...,ijtsrd,,2022-07-22

In [13]:
df_ai_filtered.columns

Index(['type', 'id', 'tags', 'intraHash', 'label', 'user', 'description',
       'date', 'changeDate', 'count', 'url', 'interHash', 'pub-type', 'year',
       'author', 'authors', 'volume', 'pages', 'bibtexKey', 'publisher'],
      dtype='object')

In [14]:
#lets only use columns which seems useful for us

useful_coulmns = ['id', 'tags', 'label', 'description', 'date', 'changeDate', 'url', 'pub-type', 'year', 'author', 'authors', 'publisher'] 

In [15]:
df_ai_filtered = df_ai_filtered[useful_coulmns]

In [16]:
#initialise pyterrier
if not pt.started():
    pt.init()

terrier-assemblies 5.8 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.8 jar not found, downloading to /root/.pyterrier...
Done


PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [17]:
#pyterrier exspects a docno field
df_ai_filtered['docno'] = df_ai_filtered['id']
df_ai_filtered['text'] = df_ai_filtered['label']

In [18]:
#now we start to index our data
#more details at https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html

index_folder = "./ai_index"

indexer = pt.DFIndexer(index_folder, overwrite=True)
index_ref = indexer.index(df_ai_filtered['text'], df_ai_filtered['docno'])

print(f"path to our index: {index_ref.toString()}")

path to our index: ./ai_index/data.properties


In [19]:
#load our index
index = pt.IndexFactory.of(index_ref)

In [20]:
print(index.getCollectionStatistics().toString())

Number of documents: 1000
Number of terms: 1377
Number of postings: 6110
Number of fields: 0
Number of tokens: 6349
Field names: []
Positions:   false



In [23]:
#term -> term_id Nt Tf
#Nt: In how many documents does the term occur
#TF: How often does the term occur in total

for kv in index.getLexicon():
    print(f"{kv.getKey()} -> {kv.getValue().toString()}")

0 -> term822 Nt=1 TF=1 maxTF=1 @{0 0 0}
02 -> term588 Nt=2 TF=2 maxTF=1 @{0 2 2}
1 -> term327 Nt=2 TF=2 maxTF=1 @{0 5 6}
10 -> term544 Nt=3 TF=3 maxTF=1 @{0 9 6}
10th -> term580 Nt=2 TF=2 maxTF=1 @{0 15 4}
11 -> term119 Nt=3 TF=3 maxTF=1 @{0 19 6}
11th -> term1130 Nt=4 TF=4 maxTF=1 @{0 25 4}
12 -> term381 Nt=2 TF=2 maxTF=1 @{0 32 4}
12th -> term1090 Nt=4 TF=4 maxTF=1 @{0 36 0}
13 -> term118 Nt=5 TF=5 maxTF=1 @{0 43 0}
13th -> term1201 Nt=3 TF=3 maxTF=1 @{0 52 2}
14 -> term526 Nt=4 TF=4 maxTF=1 @{0 57 4}
14th -> term1044 Nt=4 TF=4 maxTF=1 @{0 64 6}
15 -> term204 Nt=4 TF=4 maxTF=1 @{0 73 0}
15th -> term1053 Nt=4 TF=4 maxTF=1 @{0 78 4}
16 -> term263 Nt=5 TF=5 maxTF=1 @{0 86 0}
16th -> term281 Nt=5 TF=5 maxTF=1 @{0 94 4}
17 -> term203 Nt=4 TF=4 maxTF=1 @{0 103 4}
17th -> term328 Nt=5 TF=5 maxTF=1 @{0 111 0}
18 -> term216 Nt=4 TF=4 maxTF=1 @{0 119 4}
185 -> term807 Nt=2 TF=2 maxTF=1 @{0 127 0}
18th -> term218 Nt=6 TF=6 maxTF=1 @{0 130 0}
19 -> term385 Nt=5 TF=5 maxTF=1 @{0 140 4}
1955 -> te

In [25]:
term = "game"
index.getLexicon()[term].toString()

'term46 Nt=13 TF=14 maxTF=2 @{0 3285 6}'

In [26]:
#how often do the terms occur
term_freq_dict = {}

for kv in index.getLexicon():
    term_freq_dict[kv.getKey()] = kv.getValue().frequency

In [27]:
#sort all terms from the index in descending order 
term_freq_dict_sorted = sorted(term_freq_dict.items(), key=lambda x: x[1], reverse=True)

In [28]:
#show the 30 nost popular entries
top_k = 30
term_freq_dict_sorted[:top_k]

[('intellig', 991),
 ('artifici', 957),
 ('confer', 95),
 ('proceed', 93),
 ('system', 63),
 ('ai', 62),
 ('approach', 50),
 ('advanc', 44),
 ('us', 44),
 ('applic', 42),
 ('comput', 39),
 ('base', 37),
 ('gener', 35),
 ('learn', 35),
 ('human', 31),
 ('ki', 30),
 ('nation', 29),
 ('research', 29),
 ('modern', 27),
 ('septemb', 27),
 ('german', 25),
 ('model', 25),
 ('educ', 24),
 ('network', 23),
 ('distribut', 22),
 ('intern', 22),
 ('annual', 21),
 ('develop', 21),
 ('germani', 21),
 ('impact', 21)]

In [86]:
#define our search engine
search_engine = pt.BatchRetrieve(index, wmodel="Tf")

In [124]:
search_engine.search("system")

,qid,docid,docno,rank,score,query
0,1,321,https://www.bibsonomy.org/bibtex/24768ef52a6c0...,0,2.0,system
1,1,32,https://www.bibsonomy.org/bibtex/249f58604f738...,1,1.0,system
2,1,50,https://www.bibsonomy.org/bibtex/2f693b1d1c623...,2,1.0,system
3,1,61,https://www.bibsonomy.org/bibtex/2f75838770906...,3,1.0,system
4,1,116,https://www.bibsonomy.org/bibtex/2ee0b496413e1...,4,1.0,system
...,...,...,...,...,...,...
57,1,930,https://www.bibsonomy.org/bibtex/2c1d6d19484ec...,57,1.0,system
58,1,937,https://www.bibsonomy.org/bibtex/2ef217db6a5b4...,58,1.0,system
59,1,952,https://www.bibsonomy.org/bibtex/293fb4fd88de7...,59,1.0,system
60,1,963,https://www.bibsonomy.org/bibtex/20dac3cd23401...,60,1.0,system


So the `search()` method returns a dataframe with columns:
 - `qid`: this is by default "1", since it's our first and only query
 - `docid`: Terrier' internal integer for each document
 - `docno`: the external (string) unique identifier for each document
 - `score`: since we use the `Tf` weighting model, this score corresponds the total frequency of the query (terms) in each document
 - `rank`: A handy attribute showing the descending order by score
 - `query`: the input query

As expected, the `Tf` weighting model used here only counts the frequencies of the query terms in each document, i.e.:
$$
score(d,q) = \sum_{t \in q} tf_{t,d}
$$

In [125]:
search_engine.search("intelligent education")

,qid,docid,docno,rank,score,query
0,1,730,https://www.bibsonomy.org/bibtex/2b476e9f37ca1...,0,5.0,intelligent education
1,1,629,https://www.bibsonomy.org/bibtex/226a944b1f244...,1,3.0,intelligent education
2,1,706,https://www.bibsonomy.org/bibtex/224ac67b53792...,2,3.0,intelligent education
3,1,787,https://www.bibsonomy.org/bibtex/21c47c79fb260...,3,3.0,intelligent education
4,1,988,https://www.bibsonomy.org/bibtex/2dfb01a4c7c8d...,4,3.0,intelligent education
...,...,...,...,...,...,...
899,1,984,https://www.bibsonomy.org/bibtex/2025ecfe8993f...,899,1.0,intelligent education
900,1,985,https://www.bibsonomy.org/bibtex/28b6f3d4fdad8...,900,1.0,intelligent education
901,1,986,https://www.bibsonomy.org/bibtex/2f4c4085c258c...,901,1.0,intelligent education
902,1,993,https://www.bibsonomy.org/bibtex/2b44f329ab5e2...,902,1.0,intelligent education


In [31]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [43]:
#stem the term its base form
stem = stemmer.stem("learning")
stem

'learn'

In [44]:
#in how many documents does the stem 'learn' occur?
lexicon = index.getLexicon()
lexicon[stem].getDocumentFrequency()

33

In [45]:
#how many documents occur in our index?
index.getCollectionStatistics().numberOfDocuments

1000

In [52]:
def get_idf_for_term(term, index):
  lex = index.getLexicon()
  stemmed_term = stemmer.stem(term)

  df_term = lex[stemmed_term].getDocumentFrequency() if stemmed_term in lex else 1
  N = index.getCollectionStatistics().numberOfDocuments
  
  #inverse document frequency
  idf = N/df_term

  #apply logarithm(base 10) to idf
  log_ifd = np.emath.logn(10, idf)

  return log_ifd

In [57]:
get_idf_for_term("artificial", index)

0.04769199033787481

In [91]:
search_engine_tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")

In [126]:
search_engine_tfidf.search("intelligent education")

,qid,docid,docno,rank,score,query
0,1,276,https://www.bibsonomy.org/bibtex/201f2eb94f27f...,0,4.555594,intelligent education
1,1,770,https://www.bibsonomy.org/bibtex/2763f67bf4e7d...,1,4.555594,intelligent education
2,1,840,https://www.bibsonomy.org/bibtex/29cc0434b3e50...,2,4.555594,intelligent education
3,1,21,https://www.bibsonomy.org/bibtex/259133e526641...,3,4.209707,intelligent education
4,1,58,https://www.bibsonomy.org/bibtex/259133e526641...,4,4.209707,intelligent education
...,...,...,...,...,...,...
899,1,201,https://www.bibsonomy.org/bibtex/22113c27f2328...,899,0.322930,intelligent education
900,1,204,https://www.bibsonomy.org/bibtex/23934d5315fab...,900,0.322930,intelligent education
901,1,241,https://www.bibsonomy.org/bibtex/2201d72f3a55d...,901,0.322930,intelligent education
902,1,280,https://www.bibsonomy.org/bibtex/268ebe1d6569e...,902,0.322930,intelligent education


In [127]:
df_ai_filtered.columns

Index(['id', 'tags', 'label', 'description', 'date', 'changeDate', 'url',
       'pub-type', 'year', 'author', 'authors', 'publisher', 'docno'],
      dtype='object')

In [144]:
ai_dict = df_ai_filtered.to_dict(orient='records')

In [146]:
ai_dict

[{'id': 'https://www.bibsonomy.org/bibtex/24e338b04bc2abafd9fa0a1f4fe79103a/ijtsrd',
  'tags': ['710-714'],
  'label': 'Artificial Intelligence in Agriculture',
  'description': '',
  'date': '2021-04-13 13:15:04',
  'changeDate': '2021-04-13 13:15:04',
  'url': 'https://www.ijtsrd.com/engineering/electrical-engineering/38513/artificial-intelligence-in-agriculture/matthew-n-o-sadiku',
  'pub-type': 'article',
  'year': '2021',
  'author': ['Matthew N. O. Sadiku | Sarhan M. Musa | Abayomi Ajayi-Majebi'],
  'authors': [{'first': 'Matthew N. O. Sadiku | Sarhan M. Musa | Abayomi',
    'last': 'Ajayi-Majebi'}],
  'publisher': nan,
  'docno': 'https://www.bibsonomy.org/bibtex/24e338b04bc2abafd9fa0a1f4fe79103a/ijtsrd',
  'text': 'Artificial Intelligence in Agriculture'},
 {'id': 'https://www.bibsonomy.org/bibtex/29f3b593035714a894c265f4fb9c535e7/ijtsrd',
  'tags': ['Artificial', 'Intelligence', 'Risks', 'and', 'Benefits'],
  'label': 'Artificial Intelligence Benefit and Risks',
  'description

In [172]:
#Now we want to build an index with multiple fields

index_folder_mult = "./ai_index_mult"

fields=['docno', 'text', 'tags', 'description']

indexer_mult = pt.IterDictIndexer(index_folder_mult, meta={'docno': 200, 'text': 4096}, overwrite=True)
index_ref_mult = indexer_mult.index(ai_dict, fields=fields)

print(f"path to our index: {index_ref_mult.toString()}")

path to our index: ./ai_index_mult/data.properties


In [173]:
search_engine_mult = pt.BatchRetrieve(index_ref_mult, wmodel="TF_IDF")

In [189]:
res = search_engine_mult.search("intelligence")
res

,qid,docid,docno,rank,score,query
0,1,706,https://www.bibsonomy.org/bibtex/224ac67b53792...,0,0.935625,intelligence
1,1,994,https://www.bibsonomy.org/bibtex/2451eb56bc876...,1,0.923010,intelligence
2,1,334,https://www.bibsonomy.org/bibtex/29a73b5ea0550...,2,0.878758,intelligence
3,1,499,https://www.bibsonomy.org/bibtex/2f7cf941158e2...,3,0.878758,intelligence
4,1,653,https://www.bibsonomy.org/bibtex/2cd2e7a05ba63...,4,0.878758,intelligence
...,...,...,...,...,...,...
938,1,313,https://www.bibsonomy.org/bibtex/2b5cd3e615563...,938,0.388732,intelligence
939,1,608,https://www.bibsonomy.org/bibtex/2b4e425ae3b44...,939,0.388732,intelligence
940,1,342,https://www.bibsonomy.org/bibtex/210b214d3428a...,940,0.375925,intelligence
941,1,354,https://www.bibsonomy.org/bibtex/299846faf0a0d...,941,0.369832,intelligence


In [196]:
df_ai_filtered[df_ai_filtered['docno'] == first_id].iloc[0]['text']

'Intelligence Quotient and Intelligence Grade of Artificial Intelligence.'

In [187]:
#save our data
pickle.dump(df_ai_filtered, open("workspace/ai_publications.pkl", "wb"))